In [1]:
import glob
import os
import pandas as pd
def read_data(dirname):
    pat = "{}/*/*.[tT][xX][tT]".format(dirname)
    fs = glob.glob(pat)
    contents = []
    answers = []
    for fn in fs:
        with open(fn, "r", encoding="utf-8") as f:
            news = f.read()
        contents.append(news)
        dn = os.path.split(fn)[0]
        dn = os.path.split(dn)[-1]
        answers.append(dn)
    df = pd.DataFrame({
        "content":contents,
        "ans":answers
    })
    return df

In [2]:
train_df = read_data("chinese_news_trans")
train_df

,content,ans
0,【 日 期 】19960104\n【 版 號 】1\n【 標 題 】合巢蕪高速公路巢蕪...,交通
1,【 日 期 】19960226\n【 版 號 】5\n【 標 題 】我首臺載人磁懸浮列...,交通
2,大秦鐵路萬噸列車試運成功\n新華社北京５月２４日電（通訊員李忠民）大秦...,交通
3,遼寧省檯安縣村村都通柏油路鄉村公路建設全國數第\n一\n (4)...,交通
4,北京—烏蘭巴托—莫斯料３／４次國際旅客列車通車\n３０週年\n ...,交通
...,...,...
2632,馬玉芹破女子４００米跑全國青年紀錄\n新華社鞍山５月１０日電（通訊員王...,體育
2633,國際奧委會中國臺北委員吳經國訪問北京\n新華社北京５月１０日電記者《中...,體育
2634,亞奧理事會３９個成員組織全部以書面形式確認參加\n第十一屆亞運會\n ...,體育
2635,世界盃乒乓球團體賽男子團體採用新賽制\n新華社北京５月１０日電（記者樑...,體育


In [3]:
test_df = read_data("chinese_news_test")
test_df

,content,ans
0,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
1,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
2,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
3,三趟火車停開 乘客可全額退票 \n瀏覽次數：1180 \n 昨日，來自鐵路部門的消息說...,交通
4,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
...,...,...
96,最優秀選手無緣亞運會健美賽\n \n \n 健美在亞洲運動會上是“新生兒”——韓國釜...,體育
97,\n各國記者眼中的羽毛球世錦賽\n \n\n-----------------------...,體育
98,\n友好運動會第五天 東道主選手大顯神威\n2001年09月03日02:51:40 新華社 ...,體育
99,不靠技術比運氣\n\n 第二屆奧運會在巴黎舉行，同時這裡也正在舉行國際博覽會，東道主把一些...,體育


In [7]:
labels = train_df["ans"].unique()
label_to_idx = {l:i for i, l in enumerate(labels)}
idx_to_label = {i:l for i, l in enumerate(labels)}

In [8]:
from urllib.request import urlretrieve
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.txt.big")

('dict.txt.big', <http.client.HTTPMessage at 0xdc3c098048>)

In [9]:
import jieba
jieba.set_dictionary("dict.txt.big")
def newscut(n):
    n = n.replace("\r", "").replace("\n", "")
    return " ".join(jieba.cut(n))
x_train = train_df["content"].apply(newscut)
x_test = test_df["content"].apply(newscut)
y_train = train_df["ans"].replace(label_to_idx)
y_test = test_df["ans"].replace(label_to_idx)

Building prefix dict from C:\Users\A30\PycharmProjects\MLDemo\dict.txt.big ...
Dumping model to file cache C:\Users\A30\AppData\Local\Temp\jieba.u6b0fe39fbac1b941342ce6d335bf1d88.cache
Loading model cost 2.117 seconds.
Prefix dict has been built succesfully.


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_vec = vec.fit_transform(x_train)
x_test_vec = vec.transform(x_test)

In [13]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train_vec, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [14]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_vec)
accuracy_score(pre, y_test)

1.0

In [29]:
n = input("輸入新聞:")
n = newscut(n)
n = vec.transform([n])
proba = clf.predict_proba(n)[0]
print("*" * 30)
plist = sorted(zip(labels, proba), key=lambda x:x[1], reverse=True)
for l, p in plist:
    print(l, "類別的機率:", round(p * 100, 2), "%")

輸入新聞:陳秉立（阿翔）、謝忻6月爆發當街車吻的「偷忻不倫戀」，讓兩人形象跌落谷底，阿翔停工2個月後復出，但一回歸《綜藝大集合》，節目收視就下滑，《國光幫幫忙》則是無限期告假，他與老婆Grace愛的結晶「翔食堂水餃」，更是全面下架。   今年初，阿翔首度投資食品生意，夫妻倆推出水餃品牌「翔食堂水餃」，為了兼顧美味及食安，阿翔更親自研發口味、水餃皮的厚度，連內餡比例及來源都相當講究。為了打響第一砲，阿翔特地恩准老婆「拋頭露臉」，夫妻倆不時在鏡頭前曬恩愛賣餃子。  年初阿翔揪老婆Grace賣水餃，讓她升格老闆娘，但因「偷忻不倫戀」現已收攤。（圖／翻攝自Grace臉書） 年初阿翔揪老婆Grace賣水餃，讓她升格老闆娘，但因「偷忻不倫戀」現已收攤。（圖／翻攝自Grace臉書）   00:00 / 00:00 Loading Ad 「翔食堂水餃」推出後好評不斷，常常被搶購一空，還打進全聯福利中心及momo購物網等通路，然而就在食品副業看似順風順水之際，六月十一日，「偷忻不倫戀」曝光後，立刻波及「翔食堂水餃」的業績，如今所有通路已暫停販賣，品牌官網的產品也全面下架。本刊調查，「翔食堂水餃」的粉絲專頁從今年六月後就沒再發文，疑似是受到阿翔的不倫戀情影響。  阿翔及Grace研發半年推出「翔食堂水餃」，夫妻倆露臉賣餃子外，也讓女兒小泡芙試吃推薦。（圖／翻攝自五味食集臉書） 阿翔及Grace研發半年推出「翔食堂水餃」，夫妻倆露臉賣餃子外，也讓女兒小泡芙試吃推薦。（圖／翻攝自五味食集臉書） 2年前阿翔拿出300萬幫Grace開「小赫本手工訂製服」工作室，目前也已歇業。（圖／報系資料庫） 2年前阿翔拿出300萬幫Grace開「小赫本手工訂製服」工作室，目前也已歇業。（圖／報系資料庫） 不只水餃事業重挫，阿翔花三百萬幫Grace開的手工服工作室，在爆發偷腥前就倒了，阿翔的演藝工作也回不去了。九月間，阿翔回歸外景綜藝節目《綜藝大集合》的主持群，該節目的收視率長年輕鬆破五，但他歸隊後，不少網友揚言拒看，收視率果真數度跌到三字頭，等於流失至少二萬名觀眾。阿翔的另一個節目《國光幫幫忙》近日改版後，也不見他的蹤影。此外，阿翔還有《綜藝新時代》及《鬧著玩視界旅遊》等兩個節目，他想挽回觀眾的心，顯然還得下一番苦功。  阿翔出軌後，老婆Grace曾在臉書發文「換我為你撐傘」表達原諒。九月初，有風衣廠商相中

In [21]:
sorted([("a", 0.5), ("b", 0.2), ("c", 0.3)], 
       reverse=True,
       key=lambda x:x[1])

[('a', 0.5), ('c', 0.3), ('b', 0.2)]